In [2]:
import pandas as pd
import re

In [70]:
df = pd.read_csv('일자리정보_정리_1000건.csv')

## 데이터 정제

In [73]:
df['구인상태'].fillna('구인마감', inplace=True)
df['연령'].fillna('무관', inplace=True)
df['성별'].fillna('무관', inplace=True)
df['종류'].fillna('민간', inplace=True)
df['근무형태'].fillna('주5일', inplace=True)

In [74]:
def standardize_gender(gender):
    if gender == '남' or gender == '남성':
        return '남성'
    if gender == '여' or gender == '여성':
        return '여성'
    else:
        return '무관'

In [75]:
df['성별'] = df['성별'].apply(standardize_gender)

In [76]:
def parse_age_string(age_string):
    if not isinstance(age_string, str) or not re.search(r'\d+', age_string):
        return {'type': '무관', 'info': 'Unspecified age condition'}
    
    # '만' 나이 확인
    age_type = '만나이' if '만' in age_string else '연나이'
    
    # 숫자 추출
    numbers = re.findall(r'\d+', age_string)
    
    # 숫자가 없는 경우, 유효한 데이터가 없음을 반환
    if not numbers:
        return {'type': '무관', 'info': 'Unspecified age condition'}
    
    if numbers:
        number = int(numbers[0])  # 첫 번째 숫자 추출

    # 조건별 범위 처리
    if '이상' in age_string:
        return {'type': age_type, 'min_age': number, 'max_age': None}
    elif '이하' in age_string:
        return {'type': age_type, 'min_age': None, 'max_age': number}
    elif '미만' in age_string:
        return {'type': age_type, 'min_age': None, 'max_age': number-1}
    
    elif '~' in age_string:
        if len(numbers) == 2:
            return {'type': age_type, 'min_age': int(numbers[0]), 'max_age': int(numbers[1])}
        elif age_string.startswith('~'):
            return {'type': age_type, 'min_age': None, 'max_age': number}
        elif age_string.endswith('~'):
            return {'type': age_type, 'min_age': number, 'max_age': None}
    else:
        # 단일 나이 처리
        return {'type': age_type, 'min_age': number, 'max_age': number}

In [77]:
# 예시
age_examples = ["20세 이상", "만 30세 미만", "25세~35세", "만 18세 이하"]
parsed_ages = [parse_age_string(age) for age in age_examples]
parsed_ages

[{'type': '연나이', 'min_age': 20, 'max_age': None},
 {'type': '만나이', 'min_age': None, 'max_age': 29},
 {'type': '연나이', 'min_age': 25, 'max_age': 35},
 {'type': '만나이', 'min_age': None, 'max_age': 18}]

In [78]:
df['연령_최종'] = df['연령'].apply(parse_age_string)

In [122]:
df['연령_최종'].fillna({'type': '만나이', 'min_age': int(18), 'max_age': (70)}, inplace=True)

In [127]:
# 연령_최종 컬럼 데이터 타입 확인
print(df['연령_최종'].apply(type).value_counts())

# float 타입이 있는 경우, 딕셔너리로 변환할 수 있는지 예시 확인
print(df[df['연령_최종'].apply(lambda x: isinstance(x, float))]['연령_최종'])


연령_최종
<class 'dict'>    992
Name: count, dtype: int64
Series([], Name: 연령_최종, dtype: object)


In [126]:
# 예시: 잘못된 float 값을 딕셔너리로 변환
def fix_age_data(x):
    if isinstance(x, float):  # 잘못된 데이터 타입 처리
        return {'type': '연나이', 'min_age': None, 'max_age': None}  # 적절한 기본값 설정
    return x

df['연령_최종'] = df['연령_최종'].apply(fix_age_data)


In [146]:
def standardize_location(location):
    
    # 서울 이외 특별 지역 처리
    excluded_keywords = ['전국', '경상남도', '충청북도']
    if any(keyword in location for keyword in excluded_keywords):
        return '전국'
    if '경기' in location:
        return '경기도'
    if '서울' in location and '경기' in location:
        return '수도권'
    
    # 서울시내 모호한 지역 정보
    if '한강' in location:
        return '서울시 한강공원'
    if '서울시내' in location or '서울지역' in location or '서울' == location.strip():
        return '서울시내'

    # '구'와 '동' 추출
    match = re.search(r'(\w+구)\s*(\w*동)?', location)
    if match:
        gu = match.group(1)
        dong = match.group(2) if match.group(2) else ""
        return f'서울특별시 {gu} {dong}'.strip()
    
    return f'서울특별시 {location}'

# 근무지역 표준화 적용
df['근무지역'] = df['근무지역'].apply(standardize_location)

In [86]:
def parse_salary(salary_str):
    # 숫자와 함께 금액 단위를 추출 (예: 2,390,124원, 27만원 등)
    match = re.search(r'(\d[\d,]*)(만원|천원|원)?', salary_str.replace(',', '').replace(' ', ''))
    if match:
        amount = int(match.group(1).replace(',', ''))
        unit = match.group(2)

        if unit == '만원':
            amount *= 10000
        elif unit == '천원':
            amount *= 1000
        
        amount = int(amount)
        
        # 금액 타입 추출 (월급, 연봉, 일급, 시급)
        if '월' in salary_str:
            pay_type = '월급'
        elif '연봉' in salary_str:
            pay_type = '연봉'
        elif '일급' in salary_str:
            pay_type = '일급'
        elif '시급' in salary_str:
            pay_type = '시급'
        else:
            pay_type = '정보 없음'
    else:
        amount = None
        pay_type = '정보 없음'

    return {'amount': amount, 'type': pay_type}

In [87]:
# 금액 파싱 적용
df['parsed_salary'] = df['급여'].apply(parse_salary)
df['급여금액'] = df['parsed_salary'].apply(lambda x: x['amount'])
df['급여형태'] = df['parsed_salary'].apply(lambda x: x['type'])

df.drop(columns=['parsed_salary'], inplace=True)

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구인상태    992 non-null    object 
 1   사업장명    992 non-null    object 
 2   근무지역    992 non-null    object 
 3   직종      992 non-null    object 
 4   연령      992 non-null    object 
 5   성별      992 non-null    object 
 6   급여      992 non-null    object 
 7   근무시간    456 non-null    object 
 8   채용인원    992 non-null    object 
 9   근무내용    992 non-null    object 
 10  지원방법    992 non-null    object 
 11  지원서류    44 non-null     object 
 12  업종      142 non-null    object 
 13  종류      992 non-null    object 
 14  구인제목    992 non-null    object 
 15  근무형태    992 non-null    object 
 16  경력조건    992 non-null    object 
 17  접수마감    992 non-null    object 
 18  필요경력    27 non-null     object 
 19  전형방법    138 non-null    object 
 20  연령_최종   992 non-null    object 
 21  급여금액    936 non-null    float64
 22  급여

In [147]:
df.head()

,Unnamed: 0,구인상태,사업장명,근무지역,직종,연령,성별,급여,근무시간,채용인원,...,종류,구인제목,근무형태,경력조건,접수마감,필요경력,전형방법,연령_최종,급여금액,급여형태
0,0,구인마감,강서50플러스센터,서울특별시 강서구 등촌동,공공행정 사무원,~59세,무관,"월 2,390,124원",NaN,3명,...,공공,강서구 등촌동 공공행정 사무원 모집,주5일,무관,모집 완료 시 마감,NaN,NaN,"{'type': '연나이', 'min_age': None, 'max_age': 59}",2390124.0,월급
1,1,구인마감,종로시니어클럽,서울시내,[노인일자리_공익형]공공복지서포터즈,만65세 이상,무관,월27만원,NaN,00명,...,공공,서울시내 [노인일자리_공익형]공공복지서포터즈 모집,주5일,무관,모집 완료 시 마감,NaN,NaN,"{'type': '만나이', 'min_age': 65, 'max_age': None}",270000.0,월급
2,2,구인마감,부림산업개발,서울특별시 중구 서소문동,건물관리인,~만62세,남성,"연봉3,000만원",08:30~18:00,1명,...,공공,중구 서소문동 건물관리인 모집,주5일,무관,모집 완료 시 마감,NaN,NaN,"{'type': '만나이', 'min_age': None, 'max_age': 62}",30000000.0,연봉
3,3,구인마감,서울경찰청,서울특별시 종로구 행촌동,시설관리원,만18세 이상,남성,"월2,344,200원(정액수당 포함)",NaN,2명,...,공공,종로구 행촌동 시설관리원 모집,격일,경력,모집 완료 시 마감,NaN,NaN,"{'type': '만나이', 'min_age': 18, 'max_age': None}",2344200.0,월급
4,4,구인마감,종로노인종합복지관,서울특별시 종로구 동숭동,[노인일자리_공익형] 공공시설도우미,만65세 이상,무관,월27만원,NaN,6명,...,공공,종로구 동숭동 [노인일자리_공익형] 공공시설도우미 모집,주5일,무관,모집 완료 시 마감,NaN,NaN,"{'type': '만나이', 'min_age': 65, 'max_age': None}",270000.0,월급


In [166]:
df.to_csv('Work_Dataset_Final.csv', index=False)

In [ ]:
def parse_age_string(age_string):
    if '무관' in age_string:
        return {'age_type': '무관', 'min_age': None, 'max_age': None}

    age_type = '만나이' if '만' in age_string else '연나이'
    clean_string = re.sub(r'\s+', '', age_string)  # 공백 제거
    numbers = [int(num) for num in re.findall(r'\d+', clean_string)]

    if len(numbers) == 1:
        number = numbers[0]
        if '이상' in clean_string:
            return {'age_type': age_type, 'min_age': number, 'max_age': None}
        elif '이하' in clean_string:
            return {'age_type': age_type, 'min_age': None, 'max_age': number}
        elif '미만' in clean_string:
            return {'age_type': age_type, 'min_age': None, 'max_age': number - 1}
        else:
            # Default case if only one number and no specific direction
            return {'age_type': age_type, 'min_age': number, 'max_age': number}
    elif len(numbers) == 2:
        min_age, max_age = sorted(numbers)
        if '미만' in clean_string:
            max_age -= 1
        return {'age_type': age_type, 'min_age': min_age, 'max_age': max_age}
    else:
        return {'age_type': age_type, 'min_age': None, 'max_age': None}

# 함수 적용 및 결과 저장
df['age_info'] = df['연령'].apply(parse_age_string)
df[['age_type', 'min_age', 'max_age']] = df['age_info'].apply(pd.Series)

# 결과 출력
df[['연령', 'age_type', 'min_age', 'max_age']].head(20)

## 업종 입력

In [65]:
df = pd.read_csv('Work_Dataset_Final.csv')

In [185]:
# 데이터 결합
df['combined_text'] = df['직종'].str.cat(df[['구인제목', '근무내용']], sep=' ').str.lower()

# 분류 규칙 및 가중치 설정
categories = {
    '사무직': {'키워드': ['경리', '회계', '사무원', '사무', '인사', '디지털'], '가중치': 1},
    '보육 관련': {'키워드': ['보육', '교육', '교사', '어린이', '안전', '지도', '어린이집', '유치원'], '가중치': 1},
    '교육 관련': {'키워드': ['교육', '강사', '강의', '학습지', '교사', '지도'], '가중치': 1},
    '운전 또는 배달': {'키워드': ['배달', '배송', '택배', '운전', '택시', '지하철', '기사'], '가중치': 1},
    '방문 서비스': {'키워드': ['가정방문', '방문', '점검', '조사', '수거'], '가중치': 1},
    '청소 및 환경 미화': {'키워드': ['환경', '미화', '청소', '소독'], '가중치': 1},
    '건설 및 건축' : {'키워드': ['건설', '건축', '공사', '작업'], '가중치': 1},
    '건물 관리': {'키워드': ['건물', '시설', '관리', '유지', '경비', '아파트', '빌딩', '보안'], '가중치': 1},
    '판매 및 고객 서비스': {'키워드': ['판매', '고객', '서비스', '매장', 'A/S', '해설'], '가중치': 1},
    '요식업 및 조리': {'키워드': ['음식', '조리', '배식', '바리스타', '식당', '주방', '카운터', '홀', '서빙', '제과', '커피'], '가중치': 1},
    '보건 의료 및 사회복지 서비스': {'키워드': ['보건', '의료', '요양', '병원', '사회복지', '환자', '의사', '간호사'], '가중치': 1},
    '기술 및 엔지니어링': {'키워드': ['기술', '엔지니어', 'IT', '전기', '에너지', '기전', '기계'], '가중치': 1},
    '기타': {'키워드': ['단속', '서포터즈', '공공근로'], '가중치': 1}
}

In [199]:
def weighted_classify(job_title, combined_text):
    scores = {category: 0 for category in categories}
    
    # 직종에서 우선적으로 키워드 검색
    for category, info in categories.items():
        for keyword in info['키워드']:
            if keyword in job_title:
                scores[category] += job_title.count(keyword) * (info['가중치'] * 2)  # 직종에 더 높은 가중치 부여
            if keyword in combined_text:
                scores[category] += combined_text.count(keyword) * info['가중치']
    
    return max(scores, key=scores.get)

In [ ]:
df['업종'] = df.apply(lambda x: weighted_classify(x['직종'], x['직종'] + ' ' + x['근무내용']), axis=1)

In [110]:
# 결과 저장
df.to_csv('Work_Dataset_Final.csv', index=False)

# 사용자 예시

In [151]:
import pandas as pd
import re
import requests

df = pd.read_csv('Work_Dataset_Final.csv')

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   사업장명           1000 non-null   object 
 1   근무지역           1000 non-null   object 
 2   직종             1000 non-null   object 
 3   연령             1000 non-null   object 
 4   성별             1000 non-null   object 
 5   급여             1000 non-null   object 
 6   근무시간           462 non-null    object 
 7   채용인원           1000 non-null   object 
 8   근무내용           1000 non-null   object 
 9   지원방법           999 non-null    object 
 10  지원서류           44 non-null     object 
 11  업종             1000 non-null   object 
 12  종류             1000 non-null   object 
 13  구인제목           1000 non-null   object 
 14  근무형태           1000 non-null   object 
 15  경력조건           1000 non-null   object 
 16  접수마감           1000 non-null   object 
 17  필요경력           29 non-null     object 
 18  전형방법     

In [153]:
df[['근무지역', '직종', '업종', '성별', '연령', 'age_type', 'min_age', 'max_age']].head(20)

,근무지역,직종,업종,성별,연령,age_type,min_age,max_age
0,서울특별시 서대문구 홍은동,어린이 교통안전지도사,보육 관련,무관,만18세 이상 만65세 미만,만나이,18.0,64.0
1,서울특별시 송파구 잠실동,교통유발부담금 현장조사원,방문 서비스,무관,무관,무관,NaN,NaN
2,서울특별시 강서구,주민설문조사 현장조사원,방문 서비스,무관,만18세 이상,만나이,18.0,NaN
3,서울특별시 중구 소공동,문화유산 안전경비원,건물 관리,무관,만60세 이상,만나이,60.0,NaN
4,서울특별시 중랑구 묵제1동,시설관리 시니어인턴,건물 관리,무관,만60세 이상 만65세 미만,만나이,60.0,64.0
5,서울특별시 금천구,시설 운영관리원,건물 관리,무관,만18세 이상,만나이,18.0,NaN
6,서울특별시 동작구,안심귀가스카우트 참여자,청소 및 환경 미화,무관,만18세 이상,만나이,18.0,NaN
7,서울특별시 은평구,바리스타,요식업 및 조리,무관,만60세 이상,만나이,60.0,NaN
8,서울특별시 강서구 등촌동,공공행정 사무원,사무직,무관,~59세,연나이,NaN,59.0
9,서울시내,[노인일자리_공익형]공공복지서포터즈,청소 및 환경 미화,무관,만65세 이상,만나이,65.0,NaN


In [190]:
user1_profile = {'이름': '김영호',
             '성별': '남성',
             '생년월일': '1962-02-04',
             '거주지역': '서울특별시 영등포구 여의도동',
             '선호업종': '기술 및 엔지니어링'
                }

### 1차 : 성별 & 연령 필터링

In [191]:
from datetime import datetime

# 사용자의 생년월일과 현재 날짜를 이용하여 나이 계산

## 만나이
def calculate_full_age(birthdate):
    today = datetime.now()
    birthdate = datetime.strptime(birthdate, "%Y-%m-%d")
    full_age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return full_age

## 한국식 나이 - '연나이'로 표기
def calculate_yearly_age(birthdate):
    today = datetime.now()
    birthdate = datetime.strptime(birthdate, "%Y-%m-%d")
    yearly_age = today.year - birthdate.year + 1
    return yearly_age

# 연령 조건과 사용자 나이를 비교하는 함수
def is_age_appropriate(user_age, age_condition):
    min_age = age_condition.get('min_age')
    max_age = age_condition.get('max_age')

    if min_age is not None and user_age < min_age:
        return False
    if max_age is not None and user_age > max_age:
        return False
    return True

# 사용자 생년월일 예시
user1_profile['만나이'] = calculate_full_age(user1_profile['생년월일'])
user1_profile['연나이'] = calculate_yearly_age(user1_profile['생년월일'])

In [192]:
print(user1_profile)

{'이름': '김영호', '성별': '남성', '생년월일': '1962-02-04', '거주지역': '서울특별시 영등포구 여의도동', '선호업종': '기술 및 엔지니어링', '만나이': 62, '연나이': 63}


In [156]:
def filter_age(row, user_profile):
    age_type = row['age_type']
    min_age = row['min_age']
    max_age = row['max_age']
    
    user_age = user_profile['만나이'] if age_type == '만나이' else user_profile['연나이']

    if min_age is not None and user_age < min_age:
        return False
    if max_age is not None and user_age > max_age:
        return False
    return True

In [157]:
def filter_gender(row, user_profile):
    gender = user_profile['성별']
    if row['성별'] == '무관':
        return True
    elif row['성별'] == gender:
        return True
    else:
        return False

In [158]:
def filter_jobs(df, user_profile):
    
    # 성별 & 연령 필터링 적용 (hard)
    def is_eligible(row):
        return filter_age(row, user_profile) and filter_gender(row, user_profile)
    
    eligible_jobs = df[df.apply(is_eligible, axis=1)]
    return eligible_jobs

In [159]:
filtered_jobs = filter_jobs(df, user1_profile)
print(len(filtered_jobs))

filtered_jobs[['사업장명', '근무지역', '직종', '업종', '연령', '성별']].head(20)

649


,사업장명,근무지역,직종,업종,연령,성별
0,서울홍제초등학교,서울특별시 서대문구 홍은동,어린이 교통안전지도사,보육 관련,만18세 이상 만65세 미만,무관
1,송파구청,서울특별시 송파구 잠실동,교통유발부담금 현장조사원,방문 서비스,무관,무관
2,강서구청,서울특별시 강서구,주민설문조사 현장조사원,방문 서비스,만18세 이상,무관
3,광희문 환구단,서울특별시 중구 소공동,문화유산 안전경비원,건물 관리,만60세 이상,무관
4,중랑구 시설관리공단,서울특별시 중랑구 묵제1동,시설관리 시니어인턴,건물 관리,만60세 이상 만65세 미만,무관
5,금천구청,서울특별시 금천구,시설 운영관리원,건물 관리,만18세 이상,무관
6,동작구 대방파출소 거점,서울특별시 동작구,안심귀가스카우트 참여자,청소 및 환경 미화,만18세 이상,무관
7,은평구청 은마루카페,서울특별시 은평구,바리스타,요식업 및 조리,만60세 이상,무관
10,부림산업개발,서울특별시 중구 서소문동,건물관리인,건물 관리,~만62세,남성
11,서울경찰청,서울특별시 종로구 행촌동,시설관리원,건물 관리,만18세 이상,남성


### 2차: 지역 기반 필터링

In [99]:
df['coordinates']

0      (37.5908908122789, 126.933218715152)
1      (37.5119564733933, 127.088282780728)
2      (37.5509646154244, 126.849533759514)
3       (37.5638997702148, 126.97979400965)
4      (37.6124136763105, 127.078484387323)
                       ...                 
995     (37.483588810333, 127.032734543897)
996     (37.517331925853, 127.047377408384)
997     (37.2749769872425, 127.00892996953)
998     (37.2749769872425, 127.00892996953)
999    (37.5509646154244, 126.849533759514)
Name: coordinates, Length: 1000, dtype: object

In [174]:
api_key = 'ee980d6886b744e6b04071525dc75559'
def get_coordinates(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    response_data = response.json()
    try:
        coordinates = response_data['documents'][0]['address']
        return float(coordinates['y']), float(coordinates['x'])
    except (IndexError, KeyError, TypeError):
        return None, None

In [160]:
# 사용자 좌표 얻기
user_address = user1_profile['거주지역']
user_coordinates = get_coordinates(user_address, api_key)
user_coordinates

(37.5267849150832, 126.930167091136)

In [161]:
from math import radians, sin, cos, sqrt, atan2
def calculate_distance(coord1, coord2):
    if None in coord1 or None in coord2:
        return float('inf')
    if not isinstance(coord1, tuple) or not isinstance(coord2, tuple):
        return float('inf')  # 좌표 형태가 튜플이 아닌 경우 처리
    
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    R = 6371.0
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine 공식
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance


In [162]:
# 구 정보 추출
def get_gu(address):
    match = re.search(r'(\w+구)', address)
    return match.group(1) if match else None

user_gu = get_gu(user1_profile['거주지역'])
print(user_gu)

filtered_jobs['gu'] = filtered_jobs['근무지역'].apply(get_gu)

filtered_jobs = filtered_jobs[filtered_jobs['gu'] == user_gu]

영등포구


C:\Users\user\AppData\Local\Temp\ipykernel_7572\1076376006.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_jobs['gu'] = filtered_jobs['근무지역'].apply(get_gu)


In [165]:
print(len(filtered_jobs))
filtered_jobs[['사업장명', '근무지역', '직종', '업종', '연령', '성별']]

28


,사업장명,근무지역,직종,업종,연령,성별
41,(주)마루에이치알,서울특별시 영등포구,영업,요식업 및 조리,만60세 이상,무관
138,주식회사 주신정,서울특별시 영등포구 여의도동,수행기사/비서,운전 또는 배달,~69세,남성
175,아세아환경(주),서울특별시 영등포구 여의도동,빌딩 미화반장,청소 및 환경 미화,무관,남성
190,코오롱포레스텔 관리단,서울특별시 영등포구 여의도동,빌딩 기전실 전기기사,기술 및 엔지니어링,~64세,남성
278,코오롱포레스텔 관리단,서울특별시 영등포구 여의도동,기전실 기전기사,기술 및 엔지니어링,~68세,남성
282,(주)성진에이엠피,서울특별시 영등포구 영등포동,병원 야간 보안요원,건물 관리,~70세,남성
370,코오롱포레스텔 관리단,서울특별시 영등포구 여의도동,기전실 기전기사,기술 및 엔지니어링,~65세,남성
382,새한항업(주),서울특별시 영등포구 당산동,건물 경비원,건물 관리,~65세,남성
395,(주)성진에이엠피,서울특별시 영등포구 영등포동,건물 경비원,건물 관리,~65세,남성
426,두레시닝 주식회사,서울특별시 영등포구 양평동,건물 경비원,건물 관리,~65세,남성


In [168]:
# 필터링된 데이터에 대해 거리 계산 및 정렬
filtered_jobs['coordinates'] = filtered_jobs['근무지역'].apply(lambda x: get_coordinates(x, api_key))
filtered_jobs['distance'] = filtered_jobs['coordinates'].apply(
    lambda x: calculate_distance(user_coordinates, x) if x and all(v is not None for v in x) else float('inf')
)
filtered_jobs = filtered_jobs.sort_values(by='distance')

filtered_jobs[['사업장명', '근무지역', '직종', '업종', '연령', '성별', 'distance']]

,사업장명,근무지역,직종,업종,연령,성별,distance
845,(주)캡스텍,서울특별시 영등포구 여의도동,건물 경비원,건물 관리,~68세,남성,0.000000
138,주식회사 주신정,서울특별시 영등포구 여의도동,수행기사/비서,운전 또는 배달,~69세,남성,0.000000
175,아세아환경(주),서울특별시 영등포구 여의도동,빌딩 미화반장,청소 및 환경 미화,무관,남성,0.000000
190,코오롱포레스텔 관리단,서울특별시 영등포구 여의도동,빌딩 기전실 전기기사,기술 및 엔지니어링,~64세,남성,0.000000
278,코오롱포레스텔 관리단,서울특별시 영등포구 여의도동,기전실 기전기사,기술 및 엔지니어링,~68세,남성,0.000000
370,코오롱포레스텔 관리단,서울특별시 영등포구 여의도동,기전실 기전기사,기술 및 엔지니어링,~65세,남성,0.000000
761,주식회사 한솔텍,서울특별시 영등포구 여의도동,건물 경비원,건물 관리,~66세,남성,0.000000
759,삼부아파트관리사무소,서울특별시 영등포구 여의도동,아파트 경비원,건물 관리,~69세,남성,0.000000
282,(주)성진에이엠피,서울특별시 영등포구 영등포동,병원 야간 보안요원,건물 관리,~70세,남성,2.586660
395,(주)성진에이엠피,서울특별시 영등포구 영등포동,건물 경비원,건물 관리,~65세,남성,2.586660


In [172]:
user1_profile['선호업종'] = '기술 및 엔지니어링'
prefer = user1_profile['선호업종']

filtered_jobs = filtered_jobs[filtered_jobs['업종'] == prefer]

# 결과 출력
filtered_jobs[['사업장명', '근무지역', '직종', '업종', '연령', '성별', 'distance']]

,사업장명,근무지역,직종,업종,연령,성별,distance
190,코오롱포레스텔 관리단,서울특별시 영등포구 여의도동,빌딩 기전실 전기기사,기술 및 엔지니어링,~64세,남성,0.0
278,코오롱포레스텔 관리단,서울특별시 영등포구 여의도동,기전실 기전기사,기술 및 엔지니어링,~68세,남성,0.0
370,코오롱포레스텔 관리단,서울특별시 영등포구 여의도동,기전실 기전기사,기술 및 엔지니어링,~65세,남성,0.0


### Example 2 : 보육교사

In [189]:
user2_profile = {'이름': '최영숙 ',
             '성별': '여성',
             '생년월일': '1959-03-07',
             '거주지역': '서울특별시 송파구 잠실동',
             '선호업종': '보육 관련'
             }

In [193]:
user2_profile['만나이'] = calculate_full_age(user2_profile['생년월일'])
user2_profile['연나이'] = calculate_yearly_age(user2_profile['생년월일'])
print(user2_profile)

{'이름': '최영숙 ', '성별': '여성', '생년월일': '1959-03-07', '거주지역': '서울특별시 송파구 잠실동', '선호업종': '보육 관련', '만나이': 65, '연나이': 66}


In [200]:
# 연령, 성별 

filtered_jobs = filter_jobs(df, user2_profile)
print(len(filtered_jobs))

filtered_jobs[['사업장명', '근무지역', '직종', '업종', '연령', '성별']].head(10)

458


,사업장명,근무지역,직종,업종,연령,성별
1,송파구청,서울특별시 송파구 잠실동,교통유발부담금 현장조사원,방문 서비스,무관,무관
2,강서구청,서울특별시 강서구,주민설문조사 현장조사원,방문 서비스,만18세 이상,무관
3,광희문 환구단,서울특별시 중구 소공동,문화유산 안전경비원,건물 관리,만60세 이상,무관
5,금천구청,서울특별시 금천구,시설 운영관리원,건물 관리,만18세 이상,무관
6,동작구 대방파출소 거점,서울특별시 동작구,안심귀가스카우트 참여자,청소 및 환경 미화,만18세 이상,무관
7,은평구청 은마루카페,서울특별시 은평구,바리스타,요식업 및 조리,만60세 이상,무관
9,종로시니어클럽,서울시내,[노인일자리_공익형]공공복지서포터즈,청소 및 환경 미화,만65세 이상,무관
12,종로노인종합복지관,서울특별시 종로구 동숭동,[노인일자리_공익형] 공공시설도우미,건물 관리,만65세 이상,무관
13,서울시,서울특별시 은평구 응암동,"바리스타, 병동 동행",보건 의료 및 사회복지 서비스,만18세 이상,무관
14,금천구청,서울특별시 금천구 시흥동,CCTV관제,기술 및 엔지니어링,만18세 이상,무관


In [201]:
# 사용자 좌표 얻기
user_address = user2_profile['거주지역']
user_coordinates = get_coordinates(user_address, api_key)
print(user_coordinates)

user_gu = get_gu(user2_profile['거주지역'])
print(user_gu)

filtered_jobs['gu'] = filtered_jobs['근무지역'].apply(get_gu)

filtered_jobs = filtered_jobs[filtered_jobs['gu'] == user_gu]

(37.5119564733933, 127.088282780728)
송파구


C:\Users\user\AppData\Local\Temp\ipykernel_7572\1584318691.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_jobs['gu'] = filtered_jobs['근무지역'].apply(get_gu)


In [202]:
# 근무지역 반영
print(len(filtered_jobs))

# 필터링된 데이터에 대해 거리 계산 및 정렬
filtered_jobs['coordinates'] = filtered_jobs['근무지역'].apply(lambda x: get_coordinates(x, api_key))
filtered_jobs['distance'] = filtered_jobs['coordinates'].apply(
    lambda x: calculate_distance(user_coordinates, x) if x and all(v is not None for v in x) else float('inf')
)
filtered_jobs = filtered_jobs.sort_values(by='distance')

filtered_jobs[['사업장명', '근무지역', '직종', '업종', '연령', '성별', 'distance']]

34


,사업장명,근무지역,직종,업종,연령,성별,distance
1,송파구청,서울특별시 송파구 잠실동,교통유발부담금 현장조사원,방문 서비스,무관,무관,0.000000
434,주식회사 진성비엠씨,서울특별시 송파구 잠실동,병의원 청소원,청소 및 환경 미화,~68세,여성,0.000000
691,(주)휴브리스,서울특별시 송파구 잠실동,가사관리사,건물 관리,~76세,여성,0.000000
915,잠실아이누리어린이집,서울특별시 송파구 잠실동,보육 교사,보육 관련,무관,무관,0.000000
313,(주)소낙스앤인,서울특별시 송파구 잠실동,건물청소원,청소 및 환경 미화,~74세,여성,0.000000
936,베베스쿨어린이집,서울특별시 송파구 신천동,보육 교사,보육 관련,무관,무관,1.352057
932,한마음방문요양센터,서울특별시 송파구,사회복지사,보건 의료 및 사회복지 서비스,무관,무관,1.582618
131,서울시농수산식품공사,서울특별시 송파구,검품 보조원,건설 및 건축,60세~69세,무관,1.582618
682,(주)휴브리스,서울특별시 송파구,가사관리사,건물 관리,~76세,여성,1.582618
487,(주)휴브리스,서울특별시 송파구,가사관리사,건물 관리,~76세,여성,1.582618


In [203]:
prefer = user2_profile['선호업종']

filtered_jobs = filtered_jobs[filtered_jobs['업종'] == prefer]

# 결과 출력
filtered_jobs[['사업장명', '근무지역', '직종', '업종', '연령', '성별', 'distance']]

,사업장명,근무지역,직종,업종,연령,성별,distance
915,잠실아이누리어린이집,서울특별시 송파구 잠실동,보육 교사,보육 관련,무관,무관,0.000000
936,베베스쿨어린이집,서울특별시 송파구 신천동,보육 교사,보육 관련,무관,무관,1.352057
